In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path
import numpy as np
import random


from utils.data import make_blobs_dataset
from utils.nnet import get_device

from utils.nnet import from_gpu
from utils.eval import compute_accuracy
from hebbcl.logger import LoggerFactory
from hebbcl.model import ScaledNet2Hidden, ScaledNet2Hidden2Ctx
from hebbcl.trainer import Optimiser, train_on_blobs
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

In [2]:
from utils.data import make_trees_block, make_trees_dataset

xe,ye,fe = make_trees_block("task_a",do_shuffle=1,c_scaling = 1,exemplar= 12)

    


In [3]:
args = parser.parse_args(args=[])
data = make_trees_dataset(args)

In [8]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.cuda = False
args.device = get_device(args.cuda)[0]
# override defaults 
# args.n_features = None # todo need to change to flattened size of image
args.n_episodes = 20
args.n_features = 1730
args.lrate_sgd = 0.5
args.perform_hebb = False
args.ctx_scaling = 2
args.ctx_avg = False

# create dataset 
dataset = make_trees_dataset(args)

# instantiate model and optimiser:
model = ScaledNet2Hidden(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
# send data to gpu
x_train = torch.from_numpy(dataset["x_train"]).float().to(args.device)
y_train = torch.from_numpy(dataset["y_train"]).float().to(args.device)

# # test: create test sets
# x_a = torch.from_numpy(dataset["x_task_a"]).float().to(args.device)
# r_a = torch.from_numpy(dataset["y_task_a"]).float().to(args.device)

# x_b = torch.from_numpy(dataset["x_task_b"]).float().to(args.device)
# r_b = torch.from_numpy(dataset["y_task_b"]).float().to(args.device)

# x_both = (
#     torch.from_numpy(np.concatenate((dataset["x_task_a"], dataset["x_task_b"]), axis=0))
#     .float()
#     .to(args.device)
# )
# r_both = (
#     torch.from_numpy(np.concatenate((dataset["y_task_a"], dataset["y_task_b"]), axis=0))
#     .float()
#     .to(args.device)
# )

# f_both = torch.from_numpy(dataset["f_all"]).float().to(args.device)



# loop over data and apply optimiser
idces = np.arange(len(x_train))
for ii, x, y in zip(idces, x_train, y_train):    
    optimiser.step(model, x, y)
    # if ii % args.log_interval == 0:            
    #     if args.verbose:
    #         print(
    #             "step {}, loss: task a {:.4f}, task b {:.4f} | acc: task a {:.4f}, task b {:.4f}".format(
    #                 str(ii),
    #                 from_gpu(optimiser.loss_funct(r_a, model(x_a))).ravel()[0],
    #                 from_gpu(optimiser.loss_funct(r_b, model(x_b))).ravel()[0],
    #                 compute_accuracy(r_a, model(x_a)),
    #                 compute_accuracy(r_b, model(x_b)),
    #             )
    #         )             

print("done")


done


In [29]:
import matplotlib.pyplot as plt
[print(len(x)) for x in (list(model.parameters()))]

1730
100
100
1


[None, None, None, None]